In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import KFold
import lightgbm as lgb
import random
from gensim.models.word2vec import Word2Vec 
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
import jieba
from tqdm.notebook import tqdm as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import time
from datetime import datetime
from category_encoders import TargetEncoder
import IPython.display as ipd
from collections import *
import copy


/home/cch/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
/home/cch/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/cch/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home/cch/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/home/cch/anaconda3/lib/pytho

In [2]:
DATA_PATH = '/data/cch/weibo'
VAR_PATH = '/data/cch/hyr/weibo/var'


# Read

In [3]:
df_test_origin_weibo = pd.read_csv('%s/test.origin_weibo.csv'%DATA_PATH, sep='\t')
df_test_repost = pd.read_csv('%s/test.repost.csv'%DATA_PATH, sep='\t')
df_train_origin_weibo = pd.read_csv('%s/train.origin_weibo.csv'%DATA_PATH, sep='\t')
df_train_repost = pd.read_csv('%s/train.repost.csv'%DATA_PATH, sep='\t')
# df_user_profile = pd.read_csv('%s/user_profile.csv'%DATA_PATH, sep='\t')
df_user_profile_agg = pickle.load(open('%s/df_user_profile_agg.pickle'%VAR_PATH, 'rb'))

In [4]:
def count2idx(num):
    if 0 <= num <= 10:
        return 0
    elif 11 <= num <= 50:
        return 1
    elif 51 <= num <= 150:
        return 2
    elif 151 <= num <= 300:
        return 3
    elif num > 300:
        return 4
    if num < 0:
        return 0
df_train_origin_weibo['label'] = df_train_origin_weibo['ForwordCount'].apply(count2idx)

df_train_origin_weibo['type'] = 'train'
df_test_origin_weibo['type'] = 'test'
df_origin_weibo = pd.concat([df_train_origin_weibo, df_test_origin_weibo])

# 特征

## 文本特征

In [5]:
def generate_wv(texts):
    text_list = []
    for text in tqdm(texts):
        text_list.append(list(jieba.cut(text)))
    wv_model= Word2Vec(text_list, min_count=1, vector_size = 10, sg = 1)
    text_embed_list = []
    
    for sentence in text_list:
        word_embed_list = []
        for word in sentence:
            word_embed_list.append(wv_model.wv[word])
        text_embed_list.append(np.array(word_embed_list).mean(0))
    
    return np.array(text_embed_list)

In [6]:
def build_tfidf_svd_matrix(texts, n_output):
    """
    """
    corpus = []
    for text in texts:
        words = list(jieba.cut(str(text)))
        use_words = []
        for word in words:
            use_words.append(word)
        corpus.append(' '.join(use_words))
    tfidf_vec = TfidfVectorizer()
    tfidf_matrix = tfidf_vec.fit_transform(corpus)
    svd = TruncatedSVD(n_components=n_output, n_iter=7, random_state=42)
    tf_idf_svd = svd.fit_transform(tfidf_matrix)
    
    return tf_idf_svd

In [7]:
def get_text_condition(text, condition_list):

    
    ret = {}
    for item in condition_list:
        ret[item['name']] = 0
        for or_condition in item['condition'].split('|'):
            and_ret = True
            for and_condition in or_condition.split('&'):
                target_str = and_condition.strip()
                if target_str not in text:
                    and_ret = False
            if and_ret:
                ret[item['name']] = 1
                break
    
    return pd.Series(ret)


def get_text_hard_feature():
    global df_weibo
    
    se_text_len = df_weibo['WeiboText'].apply(len)
    se_text_len.name = 'weibotext_len'
    
    '''
        转发&点赞 疫情 特朗普|总统 视频|链接
        粉|饭 中国&金牌 台湾 历史 发展|经济 推荐
        东京&奥运
    '''
    condition_list = [
        {'name':'转发&点赞','condition':'转发&点赞'},
        {'name':'疫情','condition':'疫情'},
        {'name':'特朗普|总统','condition':'特朗普|总统'},
        {'name':'视频|链接','condition':'视频|链接'},
        {'name':'粉|饭','condition':'粉|饭'},
        {'name':'中国&金牌','condition':'中国&金牌'},
        {'name':'台湾','condition':'台湾'},
        {'name':'历史','condition':'历史'},
        {'name':'发展|经济','condition':'发展|经济'},
        {'name':'推荐','condition':'推荐'},
        {'name':'东京&奥运','condition':'东京&奥运'},
    ]
    
    df_condition = df_weibo['WeiboText'].apply(lambda x : get_text_condition(x, condition_list))
    return pd.concat([se_text_len,df_condition], 1)


In [8]:
def precision_score_cch(predictions, ground_truths):
    
    predictions, ground_truths = np.array(predictions)-1, np.array(ground_truths)-1
    y_pred = predictions
    y_true = ground_truths

    w=[1,10,50,100,300]
    n = len(y_true)
    count_r = [0 for i in range(5)]
    count = [0 for i in range(5)]
    for i in range(n):
        count[y_true[i]] += 1
        if y_pred[i] == y_true[i]:
            count_r[y_pred[i]] += 1
    sum1 = sum(w[i]*count_r[i] for i in range(5))
    sum2 = sum(w[i]*count[i] for i in range(5))
    precision = sum1/sum2
    return precision

def get_text_stacking_feature(df_weibo):
    df_train = df_weibo.query('type=="train"')
    df_test = df_weibo.query('type=="test"')
    
    hidden_feature_list = [
        'weibotext_wv_embed', 'weibotext_tfidf',#微博内容词向量/tfidf+svd降维
        'user_intro_wv_embed', 'user_intro_tfidf'#用户个性签名词向量/tfidf+svd降维
    ]
    train_x_list, test_x_list = [], []
    
    for f in hidden_feature_list:
        hidden_train = np.array(list(df_train[f]))
        hidden_test = np.array(list(df_test[f]))
        train_x_list.append(hidden_train)
        test_x_list.append(hidden_test)
        
    train_x = np.concatenate(train_x_list, 1)
    test_x = np.concatenate(test_x_list, 1)
    
#     print(train_x.shape, test_x.shape)
    train_y = np.array(df_train['label'])
    
    SEED = 42
    params = {  
        'boosting_type': 'gbdt',  
        'objective': 'multiclass',  
        'num_class': 5,  
        'metric': 'multi_error',  
        'num_leaves': 8,  
        'max_depth': 3,
        'min_data_in_leaf': 100,  
        'learning_rate': 0.01,  
        'feature_fraction': 0.8,  
        'bagging_fraction': 0.8,  
        'bagging_freq': 5,  
        'lambda_l1': 0.5,  
        'lambda_l2': 0.5,  
        'min_gain_to_split': 0.2,  
        'verbose': -1, 

        'feature_fraction_seed':SEED,
        'bagging_seed':SEED,
    } 

    def fit_lgb(train_x, train_y):
        class_weights = [1,10,50,100,300]
        trn_data = lgb.Dataset(train_x, train_y, weight=[class_weights[int(y)] for y in train_y])
        clf = lgb.train(params, 
                        trn_data, 
                        num_boost_round = 400,
                        valid_sets = [trn_data], 
                        verbose_eval = 100, 
                        early_stopping_rounds = 100
                       )
        return clf

    def eval_logit(logit, label):
        prediction = np.argmax(logit, 1)
        return precision_score_cch(prediction+1, np.array(label).astype('int')+1)

    def cross_validation_lgb(train_x, train_y, test_x):
        n_flod = 5
        folds = KFold(n_splits=n_flod, shuffle=True,random_state=SEED)
        train_x = np.array(train_x)
        train_y = np.array(train_y)
        score_train = np.zeros((len(train_x), 5))
        score_test = np.zeros((len(test_x), 5))
        trainset_score_list = []
        for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x, train_y)):

            trn_x, trn_labels = train_x[trn_idx], train_y[trn_idx]
            val_x, val_labels = train_x[val_idx], train_y[val_idx]
            model = fit_lgb(trn_x, trn_labels)
            score_train[val_idx] = model.predict(val_x)
            score_test += model.predict(test_x)/n_flod

            train_score = eval_logit(model.predict(trn_x), trn_labels)
            trainset_score_list.append(train_score)
            
        print(eval_logit(score_train, train_y))
        return score_train, score_test
    
    score_train, score_test = cross_validation_lgb(train_x, train_y, test_x)
    
    return pd.DataFrame(np.concatenate([score_train, score_test]), columns=['text_stacking_0','text_stacking_1',
                                                                           'text_stacking_2','text_stacking_3',
                                                                           'text_stacking_4'])

## 时间特征

In [9]:
def apply_weibo_creatime(x):
    date_obj = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    return pd.Series({
        'post_day':date_obj.day,
        'post_weekday':date_obj.weekday(),
        'post_month':date_obj.month, 
        'post_hour':date_obj.hour, 
        'post_minute':date_obj.minute,
        'post_year':date_obj.year,
    })


## 用户id注入target encoding

In [10]:
# pd.get_dummies(df_weibo.query('UserId == "65ba57e1fd5daa6042ca6c12f3cc9c1b031aacbe"')['label'])

In [11]:
def set_userid_target_encode(df_weibo):
    df_weibo_train = df_weibo.query('type == "train"')
    df_weibo_test = df_weibo.query('type == "test"')  
    
    
    n_flod = 5
    folds = KFold(n_splits=n_flod, shuffle=True)
    train_x = np.array(df_weibo_train['UserId'])
    train_forword_cnt = np.array(df_weibo_train['ForwordCount'])
    train_labels = np.array(np.array(pd.get_dummies(df_weibo_train['label'])))
    test_x = np.array(df_weibo_test['UserId'])
    
    target_encode_train = np.zeros((len(train_x), 6))
    target_encode_test = np.zeros((len(test_x), 6))
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x, train_labels)):
        
        trn_x, trn_cnt, trn_labels = train_x[trn_idx], train_forword_cnt[trn_idx], train_labels[trn_idx]
        val_x, val_cnt, val_labels = train_x[val_idx], train_forword_cnt[val_idx], train_labels[val_idx]
        
        for i in range(5):
            encoder = TargetEncoder()
            encoder.fit(trn_x, trn_labels[:, i])
            target_encode_train[val_idx, i:i+1] = encoder.transform(val_x)
            target_encode_test[:, i:i+1] += encoder.transform(test_x)/n_flod

        
        encoder = TargetEncoder()
        encoder.fit(trn_x, trn_cnt)
        target_encode_train[val_idx, 5:6] = encoder.transform(val_x)
        target_encode_test[:, 5:6] += encoder.transform(test_x)/n_flod
    
    column_names = ['target_encode_0', 'target_encode_1', 'target_encode_2', 'target_encode_3', 'target_encode_4', 
                    'target_encode_cnt']
    
    target_encode_score = np.concatenate([target_encode_train, target_encode_test])
    for i in range(6):
        df_weibo[column_names[i]] = list(target_encode_score[:, i])
    return df_weibo

## 转发特征

In [12]:
def apply_pass_time(a_row):
    global mp_WeiboId_create_time
    repost_date_obj = datetime.strptime(a_row['RepostDate'], '%Y-%m-%d %H:%M:%S')
    post_date_obj = datetime.strptime(mp_WeiboId_create_time[a_row['OriginWeiboId']], '%Y-%m-%d %H:%M:%S')
    return (repost_date_obj-post_date_obj).seconds

### 1h内转发特征

In [13]:
def get_repost_user_set(df_repost_1h, df_user_profile_agg, need_weibo_id_set):
    '''
    转发用户、转发文本集合
    '''
    
    
    df_user_profile_dense = df_user_profile_agg[['follow_mean', 'follow_median', 'follow_max', 'follow_min', 'follower_mean','follower_median',
                 'follower_max', 'follower_min']]
    user_profile_mean = df_user_profile_dense.mean()
    user_profile_std = df_user_profile_dense.std()

    df_user_profile_dense = copy.deepcopy(df_user_profile_dense)
    df_user_profile_dense['norm_dense'] = list(np.array((df_user_profile_dense - user_profile_mean)/user_profile_std))
    mp_userid_dense = df_user_profile_dense['norm_dense'].to_dict()
    
    repost_id_freq_set = set(df_repost_1h['RepostUserId'].value_counts()[df_repost_1h['RepostUserId'].value_counts() > 10].index)
    mp_repost_userid_idx_freq = dict([(userid,i+1) for i, userid in enumerate(set(repost_id_freq_set))])
    
    mp_repost_userid_idx = dict([(userid,i+1) for i, userid in enumerate(set(df_repost_1h['RepostUserId']))])
    default_idx = len(mp_repost_userid_idx_freq)+1
    
    df_repost_1h['repost_text_wv'] = list(generate_wv(df_repost_1h['RepostWeiboText'].fillna(' ')))
    df_repost_1h['user_dense'] = list(df_repost_1h['RepostUserId'].map(mp_userid_dense))
    
    ret_dict_list = []
    
    for id_name, df in tqdm(df_repost_1h.groupby('OriginWeiboId')):
        ret_dict_list.append({
            'WeiboId':id_name,
            'repost_weibo_repost_text_wv':[np.zeros((10,))] + list(df['repost_text_wv']),
            'repost_weibo_repost_user_id':[0]+list(df['RepostUserId'].map(mp_repost_userid_idx)),
            'repost_weibo_repost_user_id_freq':[0]+list(df['RepostUserId'].apply(lambda x:mp_repost_userid_idx_freq.get(x,default_idx))),
            'repost_weibo_set_len' : df['RepostUserId'].shape[0]+1,
            'repost_weibo_set_user_dense':[np.zeros((8,))]+list(df['user_dense'])
        })
        
    for left_id in set(need_weibo_id_set) - set(df_repost_1h['OriginWeiboId']):
        ret_dict_list.append({
            'WeiboId':left_id,
            'repost_weibo_repost_text_wv':[np.zeros((10,))],
            'repost_weibo_repost_user_id':[0],
            'repost_weibo_repost_user_id_freq':[0],
            'repost_weibo_set_len' : 1,
            'repost_weibo_set_user_dense':[np.zeros((8,))]
        })
        
    return pd.DataFrame(ret_dict_list)

In [14]:
def generate_1h_features(df_user_profile_agg, df_repost_1h, name):
    df_repost_1h = df_repost_1h.fillna('')
    user_follow_feature_list = ['follow_mean', 'follow_median', 'follow_max', 'follow_min', 'follower_mean','follower_median',
                 'follower_max', 'follower_min']
    user_follow_dict = df_user_profile_agg[user_follow_feature_list].to_dict()
    user_attribute_dict = df_user_profile_agg[['Verified', 'Gender']].to_dict()
    
    ret_dict_list = []
    for id_name, df in tqdm(df_repost_1h.groupby({'weibo':'OriginWeiboId','userid':'OriginUserId'}[name])):
        df15 = df.query('pass_time < 15*60')
        df30 = df.query('pass_time < 30*60')
        df45 = df.query('pass_time < 45*60')
        df60 = df.query('pass_time < 60*60')
        
        
        repost_other_cnt = []
        for repost_text in df['RepostWeiboText']:
            repost_text = str(repost_text)
            repost_other_cnt.append(repost_text.count('//@'))

        repost_other_cnt = np.array(repost_other_cnt)
        repost_text_len = np.array(df['RepostWeiboText'].apply(len))

        ret_dict = {
            {'weibo':'WeiboId','userid':'UserId'}[name]:id_name,
            'repost_%s_pass_time_mean'%name:df['pass_time'].mean(),
            'repost_%s_pass_time_median'%name:df['pass_time'].median(),
            'repost_%s_pass_time_min'%name:df['pass_time'].min(),
            'repost_%s_pass_time_max'%name:df['pass_time'].max(),
            'repost_%s_cnt_15_mins'%name:df15.shape[0],
            'repost_%s_cnt_30_mins'%name:df30.shape[0],
            'repost_%s_cnt_45_mins'%name:df45.shape[0],
            'repost_%s_cnt_60_mins'%name:df60.shape[0],
            'repost_%s_cnt_15_30_mins'%name:df30.shape[0]-df15.shape[0],
            'repost_%s_cnt_30_45_mins'%name:df45.shape[0]-df30.shape[0],
            'repost_%s_cnt_45_60_mins'%name:df60.shape[0]-df45.shape[0],
            
            'repost_%s_unique_repost_userid'%name:len(df['RepostUserId'].unique()),
            'repost_%s_repeat_repost_cnt'%name:df60.shape[0]-len(df['RepostUserId'].unique()),
            'repost_%s_max_user_repost'%name:max(df['RepostUserId'].value_counts()),
            'repost_%s_other_cnt_max'%name:repost_other_cnt.max(),
            'repost_%s_other_cnt_min'%name:repost_other_cnt.min(),
            'repost_%s_other_cnt_mean'%name:repost_other_cnt.mean(),
            'repost_%s_other_cnt_std'%name:repost_other_cnt.std(),
            'repost_%s_other_cnt_0'%name:len(repost_other_cnt[repost_other_cnt>0]),

            'repost_%s_text_len_max'%name:repost_text_len.max(),
            'repost_%s_text_len_min'%name:repost_text_len.min(),
            'repost_%s_text_len_mean'%name:repost_text_len.mean(),
            'repost_%s_text_len_std'%name:repost_text_len.std(),
        }
        
        #转发用户的特征聚合
        mp_user_follow_feature = defaultdict(list)
        mp_user_attribute_feature = defaultdict(list)

        for repost_userid in df['RepostUserId']:
            for user_follow_feature in user_follow_dict:
                mp_userid_feature_v = user_follow_dict[user_follow_feature]
                if repost_userid in mp_userid_feature_v:
                    mp_user_follow_feature[user_follow_feature].append(mp_userid_feature_v[repost_userid])

            for user_attribute_feature in user_attribute_dict:
                if repost_userid in user_attribute_dict[user_attribute_feature]:
                    mp_user_attribute_feature[user_attribute_feature].append(\
                        user_attribute_dict[user_attribute_feature][repost_userid])

        
        for feature_name in mp_user_follow_feature:
            ret_dict['repost_%s_%s_mean'%(name,feature_name)] = np.array(mp_user_follow_feature[feature_name]).mean()
            ret_dict['repost_%s_%s_sum'%(name,feature_name)] = np.array(mp_user_follow_feature[feature_name]).sum()
            ret_dict['repost_%s_%s_min'%(name,feature_name)] = np.array(mp_user_follow_feature[feature_name]).min()
            ret_dict['repost_%s_%s_max'%(name,feature_name)] = np.array(mp_user_follow_feature[feature_name]).max()
            ret_dict['repost_%s_%s_std'%(name,feature_name)] = np.array(mp_user_follow_feature[feature_name]).std()

                
        for feature_name in mp_user_attribute_feature:
            ret_dict['repost_%s_%s'%(name,feature_name)] = np.mean(mp_user_attribute_feature[feature_name])

        ret_dict_list.append(ret_dict)
        
    return pd.DataFrame(ret_dict_list)

# generate_1h_features(df_user_profile_agg, df_repost_1h.iloc[:1000], 'weibo')

### 转发特征分析

### 构图

In [15]:
# len(set(df_repost_1h['CurrentUserId']) & set(df_repost_1h['RepostUserId'])),\
# len(set(df_repost_1h['CurrentUserId'])), len(set(df_repost_1h['RepostUserId']))

先放弃，转发图本身是一个子图采用有偏

In [16]:
# def generate_1h_user_id_grapg_feature()
#     df_repost_1h[['CurrentUserId', 'RepostUserId']]
#     graph = defaultdict(list)
    
# df_repost_graph_feature = generate_1h_user_id_grapg_feature()

## 交叉特征

In [17]:
def cross_se(se1, se2):
    df_tmp = pd.concat([se1, se2], 1)
    name1, name2 = se1.name, se2.name
    se_cross = df_tmp.apply(lambda x:str(x[name1])+'_'+str(x[name2]), 1)
    mp_id2idx = dict([(item, i) for i, item in enumerate(set(se_cross))])
    se_cross = se_cross.map(mp_id2idx)
    se_cross.name = 'cross_%s_%s'%(name1, name2)
    return se_cross
    
def get_cross_feature():
    global df_weibo
    
    mp_userid_idx = dict([(userid, i) for i, userid in enumerate(set(df_weibo['UserId']))])

    se_userid_idx = df_weibo['UserId'].map(mp_userid_idx)#1
    se_userid_idx.name = 'userid_idx'
    
    se_cross_userid_post_hour = cross_se(se_userid_idx, df_weibo['post_hour'])#2
    se_cross_userid_post_weekday = cross_se(se_userid_idx, df_weibo['post_weekday'])#3
    
    se_repost_1h_cnt = df_weibo['repost_weibo_cnt_60_mins'].map(count2idx)
    se_repost_1h_cnt.name = 'repost_1h_cnt_idx'
    
    se_cross_repost_hour = cross_se(se_repost_1h_cnt, df_weibo['post_hour'])#4
    se_cross_repost_weekday = cross_se(se_repost_1h_cnt, df_weibo['post_weekday'])#5
    
    

    return pd.concat([se_userid_idx, se_cross_userid_post_hour, se_cross_userid_post_weekday,
                     se_cross_repost_hour, se_cross_repost_weekday], 1)

# 主流程

In [18]:
start = time.time()
df_weibo = pd.merge(df_origin_weibo, df_user_profile_agg, how='left', on='UserId')

#文本特征
df_weibo['intro'] = df_weibo['intro'].fillna(' ')

weibotext_wv_embed = generate_wv(df_weibo['WeiboText']) 
df_weibo['weibotext_wv_embed'] = list(weibotext_wv_embed) #微博内容词向量

user_intro_wv_embed = generate_wv(df_weibo['intro'])
df_weibo['user_intro_wv_embed'] = list(user_intro_wv_embed) #用户简介词向量

weibotext_tfidf = build_tfidf_svd_matrix(df_weibo['WeiboText'], 10)
df_weibo['weibotext_tfidf'] = list(weibotext_tfidf) #微博内容tfidf+svd

user_intro_tfidf = build_tfidf_svd_matrix(df_weibo['intro'], 10)
df_weibo['user_intro_tfidf'] = list(user_intro_tfidf)#用户简介tfidf+svd
df_weibo = pd.concat([df_weibo,get_text_hard_feature()],1)

df_text_stacking = get_text_stacking_feature(df_weibo)
df_weibo = pd.concat([df_weibo, df_text_stacking], 1)
logging.info('finish text features, cost:%f'%(time.time()-start))

#微博创建时间特征
df_weibo_create_time_feature = df_weibo['WeiboCreateTime'].apply(apply_weibo_creatime)
df_weibo = pd.concat([df_weibo, df_weibo_create_time_feature], 1)
logging.info('finish weibo create time features, cost:%f'%(time.time()-start))

#微博用户target encoding
df_weibo = set_userid_target_encode(df_weibo)
logging.info('finish user id target encoding features, cost:%f'%(time.time()-start))

#转发特征
mp_WeiboId_create_time = dict(list(df_weibo[['WeiboId', 'WeiboCreateTime']].values))
df_train_repost['pass_time'] = df_train_repost.apply(apply_pass_time, 1)
df_test_repost['pass_time'] = df_test_repost.apply(apply_pass_time, 1)

df_train_repost_1h = df_train_repost.query('pass_time < 3600')
df_repost_1h = pd.concat([df_train_repost_1h, df_test_repost])

df_repost_weibo_1h_agg = generate_1h_features(df_user_profile_agg, df_repost_1h, 'weibo') #1h内微博维度特征
df_repost_userid_1h_agg = generate_1h_features(df_user_profile_agg, df_repost_1h, 'userid') #1h内用户维度特征

df_weibo = pd.merge(df_weibo, df_repost_weibo_1h_agg, how='left', on='WeiboId')
df_weibo = df_weibo.fillna(-1)
df_weibo = pd.merge(df_weibo, df_repost_userid_1h_agg, how='left', on='UserId')

#转发特征聚合
df_repost_user_set_1h = get_repost_user_set(df_repost_1h, df_user_profile_agg, set(df_weibo['WeiboId']))
df_weibo = pd.merge(df_weibo, df_repost_user_set_1h, how='left', on='WeiboId')

# 交叉特征
df_cross = get_cross_feature()
df_weibo = pd.concat([df_weibo, df_cross], 1)

logging.info('finish repost features, cost:%f'%(time.time()-start))

pickle.dump(df_weibo, open('%s/df_all.pickle'%VAR_PATH, 'wb'))

  0%|          | 0/20329 [00:00<?, ?it/s]

Building prefix dict from the default dictionary ...
2021-12-09 14:53:40,520 - DEBUG - Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2021-12-09 14:53:40,522 - DEBUG - Loading model from cache /tmp/jieba.cache
Loading model cost 0.704 seconds.
2021-12-09 14:53:41,226 - DEBUG - Loading model cost 0.704 seconds.
Prefix dict has been built successfully.
2021-12-09 14:53:41,228 - DEBUG - Prefix dict has been built successfully.
2021-12-09 14:53:58,326 - INFO - collecting all words and their counts
2021-12-09 14:53:58,327 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-12-09 14:53:58,549 - INFO - PROGRESS: at sentence #10000, processed 1151302 words, keeping 80933 word types
2021-12-09 14:53:58,779 - INFO - PROGRESS: at sentence #20000, processed 2305735 words, keeping 116286 word types
2021-12-09 14:53:58,787 - INFO - collected 117061 word types from a corpus of 2339936 raw words and 20329 sentences
2021-12-09 

  0%|          | 0/20329 [00:00<?, ?it/s]

2021-12-09 14:54:30,876 - INFO - collecting all words and their counts
2021-12-09 14:54:30,878 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-12-09 14:54:30,900 - INFO - PROGRESS: at sentence #10000, processed 136451 words, keeping 685 word types
2021-12-09 14:54:30,923 - INFO - PROGRESS: at sentence #20000, processed 273297 words, keeping 685 word types
2021-12-09 14:54:30,924 - INFO - collected 685 word types from a corpus of 277786 raw words and 20329 sentences
2021-12-09 14:54:30,925 - INFO - Creating a fresh vocabulary
2021-12-09 14:54:30,929 - INFO - Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 685 unique words (100.0%% of original 685, drops 0)', 'datetime': '2021-12-09T14:54:30.929056', 'gensim': '4.1.2', 'python': '3.7.9 (default, Aug 31 2020, 12:42:55) \n[GCC 7.3.0]', 'platform': 'Linux-4.15.0-45-generic-x86_64-with-debian-stretch-sid', 'event': 'prepare_vocab'}
2021-12-09 14:54:30,929 - INFO - Word2Vec lifecycle event {'msg

Training until validation scores don't improve for 100 rounds
[100]	training's multi_error: 0.505318
[200]	training's multi_error: 0.467986
[300]	training's multi_error: 0.442151
[400]	training's multi_error: 0.413778
Did not meet early stopping. Best iteration is:
[399]	training's multi_error: 0.413624


...

  0%|          | 0/282403 [00:00<?, ?it/s]

2021-12-09 14:58:46,157 - INFO - collecting all words and their counts
2021-12-09 14:58:46,158 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-12-09 14:58:46,181 - INFO - PROGRESS: at sentence #10000, processed 113095 words, keeping 13610 word types
2021-12-09 14:58:46,203 - INFO - PROGRESS: at sentence #20000, processed 227764 words, keeping 21528 word types
2021-12-09 14:58:46,226 - INFO - PROGRESS: at sentence #30000, processed 344566 words, keeping 27907 word types
2021-12-09 14:58:46,247 - INFO - PROGRESS: at sentence #40000, processed 447057 words, keeping 33036 word types
2021-12-09 14:58:46,286 - INFO - PROGRESS: at sentence #50000, processed 697288 words, keeping 36417 word types
2021-12-09 14:58:46,308 - INFO - PROGRESS: at sentence #60000, processed 799041 words, keeping 40863 word types
2021-12-09 14:58:46,343 - INFO - PROGRESS: at sentence #70000, processed 996209 words, keeping 44688 word types
2021-12-09 14:58:46,364 - INFO - PROGRESS: at 

2021-12-09 14:59:05,969 - INFO - worker thread finished; awaiting finish of 2 more threads
2021-12-09 14:59:05,987 - INFO - worker thread finished; awaiting finish of 1 more threads
2021-12-09 14:59:05,994 - INFO - worker thread finished; awaiting finish of 0 more threads
2021-12-09 14:59:05,995 - INFO - EPOCH - 3 : training on 3828502 raw words (2693597 effective words) took 5.6s, 477724 effective words/s
2021-12-09 14:59:07,021 - INFO - EPOCH 4 - PROGRESS: at 17.28% examples, 457280 words/s, in_qsize 5, out_qsize 0
2021-12-09 14:59:08,028 - INFO - EPOCH 4 - PROGRESS: at 35.36% examples, 463445 words/s, in_qsize 5, out_qsize 0
2021-12-09 14:59:09,040 - INFO - EPOCH 4 - PROGRESS: at 54.41% examples, 467025 words/s, in_qsize 5, out_qsize 0
2021-12-09 14:59:10,042 - INFO - EPOCH 4 - PROGRESS: at 73.33% examples, 468920 words/s, in_qsize 5, out_qsize 0
2021-12-09 14:59:11,051 - INFO - EPOCH 4 - PROGRESS: at 86.45% examples, 469788 words/s, in_qsize 5, out_qsize 0
2021-12-09 14:59:11,695 -

  0%|          | 0/17213 [00:00<?, ?it/s]

/home/cch/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  
/home/cch/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
/home/cch/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
2021-12-09 15:18:19,114 - INFO - finish repost features, cost:1478.884817


## 补充交叉特征注入24H转发

In [19]:
def get_24h_cross_target_label(df_weibo, df_labels):
    
    df_weibo_copy = copy.deepcopy(df_weibo)
    df_labels_copy = copy.deepcopy(df_labels)
    target_name_list = sorted(list(set(df_labels_copy.columns) - set(['WeiboId'])))
    n_label = len(target_name_list)
    df_weibo_copy = pd.merge(df_weibo_copy, df_labels_copy, how='left', on='WeiboId')
        
    n_flod = 5
    folds = KFold(n_splits=n_flod, shuffle=True)
    train_x = np.array(df_weibo_copy.iloc[:18000]['cross_userid_idx_post_hour']).astype('str')
    train_labels = np.array(df_weibo_copy[target_name_list].iloc[:18000].values).astype('float64')
    test_x = np.array(df_weibo_copy.iloc[18000:]['cross_userid_idx_post_hour']).astype('str')
    
    target_encode_train = np.zeros((len(train_x), n_label))
    target_encode_test = np.zeros((len(test_x), n_label))
    
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x, train_labels)):
        
        trn_x, trn_labels = train_x[trn_idx], train_labels[trn_idx]
        val_x, val_labels = train_x[val_idx], train_labels[val_idx]
        
        
        for i in range(n_label):
            encoder = TargetEncoder(smoothing=0)
#             print(trn_x, trn_labels[:, i])
            encoder.fit(trn_x, trn_labels[:, i])
            target_encode_train[val_idx, i:i+1] = encoder.transform(val_x)
#             print(encoder.transform(trn_x).values)
#             print(trn_x)
#             print(trn_labels[:, i])
#             print('*'*10)
            target_encode_test[:, i:i+1] += encoder.transform(test_x)/n_flod
    
    column_names = ['target_encode_%s'%s for s in target_name_list]
    
    df_ret = pd.DataFrame()
    df_ret.index = df_weibo_copy.index
    df_ret['WeiboId'] = df_weibo['WeiboId']
    target_encode_score = np.concatenate([target_encode_train, target_encode_test])
    for i in range(len(column_names)):
        df_ret[column_names[i]] = list(target_encode_score[:, i])
    
    return df_ret
    
    
cross_target_encode_name = ['target_encode_24h_10_reposet_cnt',
       'target_encode_24h_11_reposet_cnt', 'target_encode_24h_12_reposet_cnt',
       'target_encode_24h_13_reposet_cnt', 'target_encode_24h_14_reposet_cnt',
       'target_encode_24h_15_reposet_cnt', 'target_encode_24h_16_reposet_cnt',
       'target_encode_24h_17_reposet_cnt', 'target_encode_24h_18_reposet_cnt',
       'target_encode_24h_19_reposet_cnt', 'target_encode_24h_20_reposet_cnt',
       'target_encode_24h_21_reposet_cnt', 'target_encode_24h_22_reposet_cnt',
       'target_encode_24h_23_reposet_cnt', 'target_encode_24h_24_reposet_cnt',
       'target_encode_24h_2_reposet_cnt', 'target_encode_24h_3_reposet_cnt',
       'target_encode_24h_4_reposet_cnt', 'target_encode_24h_5_reposet_cnt',
       'target_encode_24h_6_reposet_cnt', 'target_encode_24h_7_reposet_cnt',
       'target_encode_24h_8_reposet_cnt', 'target_encode_24h_9_reposet_cnt']

df_repost_24h_origin = pickle.load(open('%s/df_repost_24h_origin.pickle'%VAR_PATH, 'rb'))
df_cross_feature_target_encode_24h = get_24h_cross_target_label(df_weibo, df_repost_24h_origin)
df_weibo = pd.merge(df_weibo, df_cross_feature_target_encode_24h, how='left', on='WeiboId')

/home/cch/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/home/cch/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in Num

In [20]:
pickle.dump(df_weibo, open('%s/df_all.pickle'%VAR_PATH, 'wb'))